<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento//BDML_202401/blob/main/Modulo02/Modulo02_CV.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


<style type="text/css">

.responsive {
 width: 100%;
 height: 25%;
}

.list-group-item.active, .list-group-item.active:focus, .list-group-item.active:hover {
    z-index: 2;
    color: #fff;
    background-color: #1B175E;
    border-color: #337ab7;
}
h1, h2, h3, h4 {
    color: #000002;
    background-color: #1B175E;
    background-image:
      linear-gradient(to right,
       #fff, #ffff00
     );

}

h1, h2, h3, h4, p {
    color: #000002;
}

a {
    color: #1B175E;
}
</style>

# Introduction

The concept behind resampling techniques for evaluating model performance is straightforward: a portion of the data is used to train the model, while the remaining data is used to assess the model's accuracy. 

This process is repeated several times with different subsets of the data, and the results are averaged and summarized. The primary differences between resampling techniques lie in the method by which the subsets of data are selected. 

In the following sections, we will discuss the main types of resampling techniques.




# Predicting Wages

Our objective today is to construct a model of individual wages

$$
w = f(X) + u 
$$

where w is the  wage, and X is a matrix that includes potential explanatory variables/predictors. In this problem set, we will focus on a linear model of the form

\begin{align}
 ln(w) & = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p  + u 
\end{align}

were $ln(w)$ is the logarithm of the wage.

To illustrate I'm going to use a sample of the NLSY97. The NLSY97 is  a nationally representative sample of 8,984 men and women born during the years 1980 through 1984 and living in the United States at the time of the initial survey in 1997.  Participants were ages 12 to 16 as of December 31, 1996.  Interviews were conducted annually from 1997 to 2011 and biennially since then.  

Let's load the packages and the data set:

In [ ]:
#install.packages("pacman") for #google colab

In [1]:
#packages
require("pacman")
p_load("tidyverse","stargazer")

nlsy <- read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/nlsy97.csv')


Loading required package: pacman

Rows: 1266 Columns: 994
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl (994): lnw_2016, educ, black, hispanic, other, exp, afqt, mom_educ, dad_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


what are the predictors that I have available?

In [ ]:
glimpse(nlsy) 

Let's keep a couple of these predictors and look at their statistics

In [2]:
nlsy<- nlsy  %>% select(lnw_2016, 
                        educ,
                        black,
                        hispanic,
                        other,
                        exp,
                        afqt,
                        mom_educ,
                        dad_educ)

The descriptive statistics?

In [3]:
stargazer(data.frame(nlsy), header=FALSE, type='text',title="Variables Included in the Selected Data Set")


Variables Included in the Selected Data Set
Statistic   N    Mean  St. Dev.  Min    Max  
---------------------------------------------
lnw_2016  1,266 3.110   0.869   -1.955 7.393 
educ      1,266 14.314  2.900     8      20  
black     1,266 0.182   0.386     0      1   
hispanic  1,266 0.117   0.321     0      1   
other     1,266 0.007   0.084     0      1   
exp       1,266 14.582  3.238     7      23  
afqt      1,266 5.325   2.915   0.000  10.000
mom_educ  1,266 13.303  2.401     4      20  
dad_educ  1,266 13.254  2.908     3      20  
---------------------------------------------


# Validation Set  Approach

The first method to evaluate out-of-sample performance is the validation set approach. In this approach, a fixed portion of the data is designated as the validation set, and the model is trained on the remaining data. The model's performance is then evaluated on the validation set. These partitions are usually called:

   - Training sample: to build/estimate/train the model
   - Testing (validation, hold-out) sample:  to evaluate its performance 

Partitions can be of any size. Usually, 70%-30% or 80%-20% are used. Graphically, a 70%-30% partition looks like:     
    
<div>
<img src="figs_notebook_SS03/30-70.png" width="500"/>
</div>

Let's implement this in `R`.

We begin by generating a sample index that will indicate with `TRUE` those observations randomly assigned to the training data set with 70% probability, and with `FALSE` those observations randomly assigned to the testing data set with 30% chance.

In [4]:
#Make this example reproducible
set.seed(123)

#use 70% of the dataset as a training set and 30% as a test set
sample <- sample(c(TRUE, FALSE), nrow(nlsy), replace=TRUE, prob=c(0.7,0.3))
head(sample)

[1]  TRUE FALSE  TRUE FALSE FALSE  TRUE

we can check that the partition:

In [5]:
sum(sample)/nrow(nlsy)

[1] 0.706951

With the above index, we generate the partition:

In [6]:
train  <- nlsy[sample, ] #train sample those that are TRUE in the sample index
test   <- nlsy[!sample, ] #test sample those that are FALSE in the sample index
dim(train)

[1] 895   9

Another way of partitioning the data is to use `dplyr` package included in `tidyverse`. With this approach, we first create a column with the row number, take a sample of 70% of the sample and use this sample for training. The test set is the remaining, obtained by `anti_join()`. 

In [ ]:
#remove the comments to execute this portion of the code
#Make this example reproducible
#set.seed(123)

#create ID column with a row number
#nlsy$id <- 1:nrow(nlsy)

#use 70% of the dataset as a training set and 30% as a test set 
#train <- nlsy %>% dplyr::sample_frac(0.70)
#test  <- dplyr::anti_join(nlsy, train, by = 'id')



## Predicting wages

With these partitions in place, we can start building our predictive models. We begin by using a simple model with no covariates, just a constant:

\begin{align}
 ln(w) & = \beta_0  + u 
\end{align}

In [7]:
model1<-lm(lnw_2016~1,data=train)
summary(model1)


Call:
lm(formula = lnw_2016 ~ 1, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.0668 -0.4639 -0.0127  0.4138  4.2818 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.11143    0.02847   109.3   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8519 on 894 degrees of freedom


In this case, the prediction for the log wage is the average train sample average

$$
\hat{ ln(w)}=\hat{\beta_1}=\frac{\sum  ln(w)_i}{n}=m
$$

In [8]:
coef(model1)

(Intercept) 
   3.111427

In [9]:
paste("Coef:", mean(train$lnw_2016))

[1] "Coef: 3.11142668361229"

Since we are concerned with predicting well out-of -sample, we need to evaluate our model in the testing data. For that, we use the coefficient estimated in the training data and use it as a predictor in the testing data:

In [10]:
#prediction on new data
test$hat_lnw_m1<-predict(model1,newdata = test)

Then we can calculate the out-of-sample performance using the MSE:

$$
test\,MSE=E(( ln(w)-\hat{ ln(w)})^2)
$$ 
in `R`:

In [11]:
with(test,mean((lnw_2016-hat_lnw_m1)^2))

[1] 0.8276081

This is quite a naive model that uses the sample average as a prediction. 

Let's see if we can improve (reduce the prediction error) this model.

To improve our prediction, we can start adding explanatory variables. Let's begin by adding only one variable, `education (educ)`:


\begin{align}
 ln(w) & = \beta_0 + \beta_1 Educ + u 
\end{align}

In [12]:
model2<-lm(lnw_2016~educ,data=train)
test$hat_lnw_m2<-predict(model2,newdata = test)

and evaluate the  out-of-sample performance

In [13]:
with(test,mean((lnw_2016-hat_lnw_m2)^2))

[1] 0.7963705

There's a clear diminution in MSE. Let's add complexity by adding more variables:



\begin{align}
 ln(w) & = \beta_0 + \beta_1 Educ   + \beta_2 Exp  + \beta_3 Afqt + \beta_4 Mom_Educ + \beta_5 Dad_Educ  + u 
\end{align}

In [14]:
model3<-lm(lnw_2016~educ + exp + afqt + mom_educ + dad_educ,data=train)
test$hat_lnw_m3<-predict(model3,newdata = test)


The performance:

In [15]:
with(test,mean((lnw_2016-hat_lnw_m3)^2))

[1] 0.7881659

In this case, the MSE keeps improving. Is there a limit to this improvement? Can we keep adding features and complexity? What about an extremely complex model that includes polynomials and interactions.

In [16]:
model4<-lm(lnw_2016~poly(educ,8,raw=TRUE):poly(exp,3,raw=TRUE):
           poly(afqt,3,raw=TRUE)+ poly(mom_educ,3,raw=TRUE):
           poly(dad_educ,3,raw=TRUE) +
           black+hispanic + other,data=train)

In [17]:
summary(model4)


Call:
lm(formula = lnw_2016 ~ poly(educ, 8, raw = TRUE):poly(exp, 3, 
    raw = TRUE):poly(afqt, 3, raw = TRUE) + poly(mom_educ, 3, 
    raw = TRUE):poly(dad_educ, 3, raw = TRUE) + black + hispanic + 
    other, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.3946 -0.4240 -0.0107  0.3864  4.4914 

Coefficients: (3 not defined because of singularities)
                                                                                  Estimate
(Intercept)                                                                      1.678e+00
black                                                                           -3.327e-01
hispanic                                                                         9.667e-02
other                                                                           -6.667e-01
poly(mom_educ, 3, raw = TRUE)1:poly(dad_educ, 3, raw = TRUE)1                    8.091e-02
poly(mom_educ, 3, raw = TRUE)2:poly(dad_educ, 3, raw = TRUE)1                

In [18]:
test$hat_lnw_m4<-predict(model4,newdata = test)

The performance:

In [19]:
with(test,mean((lnw_2016-hat_lnw_m4)^2))

[1] 0.9338882

Le'ts put all these performance resuls in a table and compare them


In [20]:
#create vars with all the MSE
mse1<-with(test,round(mean((lnw_2016-hat_lnw_m1)^2),3))
mse2<-with(test,round(mean((lnw_2016-hat_lnw_m2)^2),3))
mse3<-with(test,round(mean((lnw_2016-hat_lnw_m3)^2),3))
mse4<-with(test,round(mean((lnw_2016-hat_lnw_m4)^2),3))

#put them in a vector
mse<-c(mse1,mse2,mse3,mse4)

#create a data frame
db<-data.frame(model=factor(c("model1","model2","model3","model4"),ordered=TRUE),
               MSE=mse)

db

model,MSE
<ord>,<dbl>
model1,0.828
model2,0.796
model3,0.788
model4,0.934


It is clear that as complexity increases, performance improves until a point where too much complexity results in inferior performance. 

This is an illustration of the Bias-Variance-Trade-Off.

Although the validation set approach is quite nice, there are at least two problems with it
  
  1. The first one is that given an original data set if part of it is left aside to test the model, fewer data is left for estimation (leading to less efficiency).
  2. A second problem is deciding which data will be used to train the model and which one to test it:
  
  <div>
<img src="figs_notebook_SS03/fig52.png" width="800"/>
</div>




# Leave-One-Out Cross-Validation (LOOCV) 

This method is similar to the Validation Set Approach, but it tries to address the latter's disadvantages. Leave-One-Out Cross-Validation (LOOCV) is a resampling technique for evaluating model performance. Each sample in the data is used once as the validation set, and the model is trained on the remaining samples. 

Graphically the LOOCV looks like this: 


<div>
<img src="figs_notebook_SS03/1.png" width="500"/>
</div>

<div>
<img src="figs_notebook_SS03/2.png" width="500"/>
</div>


<div>
<img src="figs_notebook_SS03/3.png" width="500"/>
</div>

.

.

.

.

.

.

.

.

<div>
<img src="figs_notebook_SS03/20.png" width="500"/>
</div>


LOOCV is computationally expensive, as a separate model has to be fit `n` times, where `n` is the number of samples in the data. However, LOOCV is more thorough in its model evaluation, as each sample is used as the validation set exactly once, giving a more comprehensive assessment of the model's performance.

The LOOCV estimate for the test MSE is

\begin{align}
LOOCV(n) &= \frac{1}{n}\sum MSE_{-i} \\ 
      &= \frac{1}{n}\sum(y_i -\hat{y}_{-i})^2
\end{align}

where $-i$ indicates that the model to obtain the prediction was trained in all observations except $i$.

LOOCV is particularly useful in cases where the number of samples in the data is small, and the risk of overfitting is high.


LOOCV is a special case of k-fold cross-validation, where k is equal to the number of samples in the data. Given that it's a particular case of k-fold cross-validation, we will implement this instead.





# K-Fold Cross-Validation

K-Fold Cross-Validation  is a widely used resampling technique for evaluating model performance. It involves dividing the data into k equally sized folds, where k is a user-defined constant. The model is then fit k times, with each fold used once as the validation set and the remaining k-1 folds used as the training set. This process results in k estimates of the model's performance, which can then be averaged to obtain an overall estimate. Graphically it looks like this:




<div>
<img src="figs_notebook_SS03/fold.png" width="500"/>
</div>

K-Fold Cross-Validation is a trade-off between the computational efficiency of the validation set approach and the thoroughness of LOOCV:

- K-Fold Cross-Validation provides a more robust evaluation of the model's performance than the validation set approach.  In the validation set approach, a fixed portion of the data is used as the validation set, which can result in a suboptimal estimation of the model's performance. In contrast, K-Fold Cross-Validation ensures that each sample is used in the validation set exactly once, providing a more comprehensive assessment of the model's performance.


- K-Fold Cross-Validation, on the one hand, is more computationally efficient than LOOCV, as the model is fit k times instead of n times.  On the other hand, K-Fold Cross-Validation is less thorough than LOOCV, as each sample is used in the validation set k-1/k of the time, giving a less comprehensive assessment of the model's performance.

However, K-Fold Cross-Validation is widely used in practice. It provides a good balance between computational efficiency and thoroughness while allowing the user to control the number of times the model fits.


To sum up, to implement K-Fold Cross-Validation, we need to:

- Split the data into K parts $(n=\sum_{j=1}^k n_j)$

- Fit the model leaving out one of the folds $\rightarrow$ $\hat{y}_{-k}$
  
- Cycle through all k folds
 
-  The CV(k) estimate for the test MSE is


\begin{align}
CV_{(k)} &= \frac{1}{k}\sum_{j=1}^k MSE_j \\
         &= \frac{1}{k}\sum_{j=1}^k (y_j^k-\hat{y}_{-k})^{2}
\end{align}




<div>
<img src="figs_notebook_SS03/fig54.png" width="800"/>
</div>



Let's implement it in `R`

### Splitting the data into K folds

In [21]:
#Make this example reproducible
set.seed(123)

# Specify the number of folds for
# 5-fold cross-validation
K <- 5

In [22]:
nrow(nlsy)/K

[1] 253.2

In [23]:
#generate an Index for the fold
nlsy<-nlsy  %>% mutate(fold=c(rep(1,254),
                              rep(2,253),
                              rep(3,253),
                              rep(4,253),
                              rep(5,253)))

In [24]:
table(nlsy$fold)


  1   2   3   4   5 
254 253 253 253 253 

In [26]:
head(nlsy,300)

lnw_2016,educ,black,hispanic,other,exp,afqt,mom_educ,dad_educ,fold
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.0768976,16,0,0,0,11,7.0724,12,12,1
3.2941382,9,0,0,0,19,4.7481,9,10,1
2.8308961,9,0,1,0,22,1.1987,12,9,1
4.3064590,16,0,0,0,13,8.9321,16,18,1
5.9914646,16,0,1,0,15,2.2618,16,16,1
4.7119241,16,0,0,0,14,8.9502,18,20,1
3.4299397,17,0,0,0,14,6.4555,12,16,1
3.7578723,16,0,0,0,14,9.0180,12,12,1
4.5941410,17,1,0,0,13,4.5519,16,9,1


### Fit the model leaving out one of the folds 

In [27]:
fit1<- lm(lnw_2016~educ, data= nlsy  %>% filter(fold!=1))
yhat1<- predict(fit1,newdata=nlsy  %>% filter(fold==1) )


In [28]:
# In a loop

db_train<-list()
db_test<-list()


for(i in 1:5){
    db_train[[i]]<-nlsy  %>% filter(fold!=i) # trains
    db_test[[i]]<-nlsy  %>% filter(fold==i) # tests
    
    fit<-lm(lnw_2016~educ, data=db_train[[i]])
    db_test[[i]]<- db_test[[i]]  %>% mutate(lnw_hat=predict(fit1,newdata=db_test[[i]]))
}



In [30]:
head(db_test[[2]])

lnw_2016,educ,black,hispanic,other,exp,afqt,mom_educ,dad_educ,fold,lnw_hat
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.503997,18,0,0,0,9,6.7504,12,14,2,3.308481
4.488780,19,0,0,0,11,6.7764,16,16,2,3.378024
2.922196,14,0,0,0,17,7.2445,17,14,2,3.030309
3.292984,16,0,0,0,12,3.2800,12,12,2,3.169395
3.342928,18,0,0,0,9,6.0376,18,17,2,3.308481
3.273936,16,1,0,0,13,3.2655,13,12,2,3.169395



### Calculating the MSE


Finally, we need to calculate the  CV(k) estimate for the test MSE. We know that it takes the form:

\begin{align}
CV_{(k)} &= \frac{1}{k}\sum_{j=1}^k MSE_j 
\end{align}

To implment this formula we first need to calculate the MSE for each fold:

In [31]:
MSE<-list()

for(i in 1:5){    
    MSE[[i]]<-mean((db_test[[i]]$lnw_2016- db_test[[i]]$lnw_hat)^2)
}

MSE

[[1]]
[1] 0.5798615

[[2]]
[1] 0.5391549

[[3]]
[1] 0.8745535

[[4]]
[1] 0.5965072

[[5]]
[1] 0.9531883

In [32]:
MSE<-do.call(rbind,MSE)
MSE

0.5798615
0.5391549
0.8745535
0.5965072
0.9531883


In [33]:
mean(MSE)

[1] 0.7086531

We can compare the results to that obtained using only the validation set approach:

In [34]:
db$MSE[db$model=="model2"]

[1] 0.796


# If we have enough data


In some cases, where there's enough data, researchers may use both K-Fold Cross-Validation and the validation set approach in combination to evaluate the performance of a machine learning model. This can be useful when a researcher wants to obtain a more robust evaluation of the model's performance while maintaining computational efficiency.

The following figure shows the strategy followed by Kleinberg et al. (2017) in their paper "Human decisions and machine predictions":



<div>
<img src="figs_notebook_SS03/human_decisions.png" width="500"/>
</div>


This strategy prevents the machine learning algorithm from appearing to do well simply because it is being evaluated on data it has already seen. Moreover, they add an extra layer of protection to ensure that the results are not an artifact of unhelpful "human data mining," adding a "pure hold-out."

By combining K-Fold Cross-Validation and the validation set approach, researchers can obtain a more comprehensive evaluation of the model's performance while maintaining computational efficiency. The specific combination of K-Fold Cross-Validation and the validation set approach will depend on the researcher's goals and the particular constraints of the study. When choosing a resampling technique, it is essential to carefully consider the trade-offs between computational efficiency and thoroughness.




## Extra

### The `lapply()` version

In [ ]:
#Make this example reproducible
set.seed(123)

# Specify the number of folds for
# 5-fold cross-validation
K <- 5

#Split the data set into 5 folds
index <- split(1:nrow(nlsy), 1: K)

We used the `split` function to generate a list with indexes that will help split the dataset into 5 parts of roughly equal size. We can see the first six indexes of the observations randomly assigned to the first partition or fold


In [ ]:
head(index[[1]])

Given that the size is not divisible by 5 `R` sometimes us a *Warning*, we can verify the length of each partition:

In [ ]:
lapply(index,length)

All partitions have roughly the same size. The only partition with one extra observation is the first one.

Note that to obtain the length, we used the `lapply` function. The `lapply` function is an extremely powerful function  used to apply a function to each element of a list and return a list as a result. It stands for "list apply," and it is a commonly used function in R for working with lists and vectors. 

The basic syntax of the lapply function is: `lapply(X, FUN)`, where X is the list or vector that we want to apply the function FUN to, and FUN is the function we want to apply. FUN can be any R function, and it takes a single argument, which will be one of the elements of X. The `lapply` function returns a list, where each element of the list results from applying FUN to a corresponding element of X. In our example above, the list is the `index` element, and the function is `length`, and it returns a list with the lengths of each partition.

The `lapply` function is useful when we need to perform the same operation on multiple elements of a list or vector and return a list as a result. It is a convenient alternative to using a for loop, as it is easier to read and write, more efficient for large lists, and faster and easily parallelizable in many circumstances.

With the indices, we can then split the data set

In [ ]:
#aplicar a la lista de folds 1,2,3,4,5
splt <- lapply(1:K, function(ind) nlsy[index[[ind]], ])

Then the first partition of the matchadata set will be in the first element of the `splt` element:

In [ ]:
splt[[1]]

## Fitting the model leaving out one of the folds 


To fit the model, we will again leverage the power of `lapply`, but first we need the `rbindlist` available in the `data.table` package

In [ ]:
p_load(data.table)

m1 <- lapply(1:K, function(ii) lm(lnw_2016~educ, data = rbindlist(splt[-ii]))) 

In the previous step, we fitted the model `lnw_2016~educ` in every fold except the `ii` fold. We achieved this by creating a function that runs a linear regression on a data set constructed by binding all the elements in the `splt` list except the element indexed by `ii`.

Next, we fit the model in the fold that was left as a testing fold.

In [ ]:
p1 <- lapply(1:K, function(ii) data.frame(predict(m1[[ii]], newdata = rbindlist(splt[ii]))))

We can see then that it created a vector with predictions:

In [ ]:
p1[1]

Now we bind this vector to each fold so we have extra column in each fold with the prediction, named `yhat`, 

In [ ]:
for (i in 1:K) {
  colnames(p1[[i]])<-"yhat" #change the name
  splt[[i]] <- cbind(splt[[i]], p1[[i]])

}

In [ ]:
splt[[1]]


### Calculating the MSE


Finally, we need to calculate the  CV(k) estimate for the test MSE. We know that it takes the form:

\begin{align}
CV_{(k)} &= \frac{1}{k}\sum_{j=1}^k MSE_j
\end{align}

To implment this formula we first need to calculate the MSE for each fold using `lapply`:

In [ ]:
MSE2_k <- lapply(1:K, function(ii) mean((splt[[ii]]$lnw_2016 - splt[[ii]]$yhat)^2))
                 
MSE2_k

And finally, calculate the mean:

In [ ]:
mean(unlist(MSE2_k))

Note that since `lapply` always return a list, to be able to average the results, we first unlisted the object and then calculated the `mean`.

Finally, we can compare the results to that obtained using only the validation set approach:

In [ ]:
db$MSE[db$model=="model2"]